## Импорт зависимостей

In [1]:
!pip install medcam3d grad-cam -q

In [107]:
import os
import torch
import torchvision
import pandas as pd
from torch import nn, optim
from transformers import GPT2Tokenizer, GPT2LMHeadModel, BertTokenizer, BertForSequenceClassification
from pytorch_grad_cam import GradCAM
from sklearn.preprocessing import MultiLabelBinarizer

## Загрузка и предобработка данных

In [108]:
rpt = pd.read_csv('/kaggle/input/chest-xrays-indiana-university/indiana_reports.csv')
proj = pd.read_csv('/kaggle/input/chest-xrays-indiana-university/indiana_projections.csv')

In [109]:
df = proj.merge(rpt, on='uid', how='left')
df = df.dropna(subset=['findings','impression'])
df['report'] = df['findings'].fillna('') + ' ' + df['impression'].fillna('')

In [110]:
df["path"] = '/kaggle/input/chest-xrays-indiana-university/images/images_normalized/' + df["filename"]

In [111]:
df.head()

,uid,filename,projection,MeSH,Problems,image,indication,comparison,findings,impression,report,path
0,1,1_IM-0001-4001.dcm.png,Frontal,normal,normal,Xray Chest PA and Lateral,Positive TB test,None.,The cardiac silhouette and mediastinum size ar...,Normal chest x-XXXX.,The cardiac silhouette and mediastinum size ar...,/kaggle/input/chest-xrays-indiana-university/i...
1,1,1_IM-0001-3001.dcm.png,Lateral,normal,normal,Xray Chest PA and Lateral,Positive TB test,None.,The cardiac silhouette and mediastinum size ar...,Normal chest x-XXXX.,The cardiac silhouette and mediastinum size ar...,/kaggle/input/chest-xrays-indiana-university/i...
2,2,2_IM-0652-1001.dcm.png,Frontal,Cardiomegaly/borderline;Pulmonary Artery/enlarged,Cardiomegaly;Pulmonary Artery,"Chest, 2 views, frontal and lateral",Preop bariatric surgery.,None.,Borderline cardiomegaly. Midline sternotomy XX...,No acute pulmonary findings.,Borderline cardiomegaly. Midline sternotomy XX...,/kaggle/input/chest-xrays-indiana-university/i...
3,2,2_IM-0652-2001.dcm.png,Lateral,Cardiomegaly/borderline;Pulmonary Artery/enlarged,Cardiomegaly;Pulmonary Artery,"Chest, 2 views, frontal and lateral",Preop bariatric surgery.,None.,Borderline cardiomegaly. Midline sternotomy XX...,No acute pulmonary findings.,Borderline cardiomegaly. Midline sternotomy XX...,/kaggle/input/chest-xrays-indiana-university/i...
6,4,4_IM-2050-1001.dcm.png,Frontal,"Pulmonary Disease, Chronic Obstructive;Bullous...","Pulmonary Disease, Chronic Obstructive;Bullous...","PA and lateral views of the chest XXXX, XXXX a...",XXXX-year-old XXXX with XXXX.,None available,There are diffuse bilateral interstitial and a...,1. Bullous emphysema and interstitial fibrosis...,There are diffuse bilateral interstitial and a...,/kaggle/input/chest-xrays-indiana-university/i...


In [112]:
# Определите имя столбца с путями к изображениям
image_col = 'path' if 'path' in df.columns else ('filename' if 'filename' in df.columns else None)
if image_col is None:
    raise KeyError("Не найден столбец с путями к изображениям. Проверьте df.columns")


Разбиение на выборки

In [113]:
from sklearn.model_selection import GroupShuffleSplit

In [114]:
split = GroupShuffleSplit(test_size=0.2, n_splits=1, random_state=42)
train_idx, test_idx = next(split.split(df, groups=df['uid']))
train_df, test_df = df.iloc[train_idx], df.iloc[test_idx]

In [115]:
train_df.shape

(5164, 12)

In [116]:
test_df.shape

(1293, 12)

In [118]:
mlb = MultiLabelBinarizer()
all_labels = df['MeSH'].explode().unique().tolist()
mlb.fit([all_labels])

MultiLabelBinarizer()

In [120]:
label_map = {
    uid: torch.tensor(mlb.transform([grp['MeSH'].tolist()])[0], dtype=torch.float)
    for uid, grp in df.groupby('uid')
}

In [122]:
tfm = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])

### DataLoader с фронтальными и латеральными проекциями

In [123]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms

In [124]:
class IUDataset(Dataset):
    def __init__(self, df, img_dir, label_map, transform=None):
        self.transform = transform
        self.groups = df.groupby('uid')
        self.uids = list(self.groups.groups.keys())
        self.img_dir = img_dir
        self.label_map = label_map

    def __len__(self):
        return len(self.uids)

    def __getitem__(self, idx):
        uid = self.uids[idx]
        group = self.groups.get_group(uid)
        # выбираем frontal
        if 'projection' in group.columns and 'frontal' in group['projection'].values:
            row = group[group['projection']=='frontal'].iloc[0]
        else:
            row = group.iloc[0]

        path = row['path']  # убедитесь, что колонка называется именно так
        image = Image.open(os.path.join(self.img_dir, path)).convert('RGB')
        if self.transform:
            image = self.transform(image)

        labels = self.label_map[uid]        # теперь это dict, а не Compose
        report = row['report']

        return {'img_f': image, 'report': report, 'labels': labels}


In [125]:
tfm = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [135]:
train_ds = IUDataset(train_df, '/kaggle/input/chest-xrays-indiana-university/images/images_normalized', transform=tfm) #, label_map=label_map)
train_dl = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=2)

test_ds = IUDataset(test_df, '/kaggle/input/chest-xrays-indiana-university/images/images_normalized', transform=tfm, label_map=label_map)
test_dl = DataLoader(test_ds, batch_size=16, shuffle=True, num_workers=2)

TypeError: IUDataset.__init__() missing 1 required positional argument: 'label_map'

In [136]:
for epoch in range(1, num_epochs + 1):
    train_loss = train_epoch(model, train_dl, criterion, optimizer)
    val_loss   = validate_epoch(model, val_dl, criterion)
    print(f"Epoch {epoch}: Train {train_loss:.4f}  Val {val_loss:.4f}")


ValueError: Target size (torch.Size([16, 1635])) must be the same as input size (torch.Size([16, 14]))

## Определение DenseTagger (CheXNet‑backbone)

In [137]:
class DenseTagger(nn.Module):
    def __init__(self, num_labels):
        super().__init__()
        self.backbone = torchvision.models.densenet121(pretrained=True)
        # Заменяем классификатор на multi-label
        in_features = self.backbone.classifier.in_features
        self.backbone.classifier = nn.Linear(in_features, num_labels)

    def forward(self, x):
        # x: [batch, 3, H, W]
        logits = self.backbone(x)
        return logits  # BCEWithLogitsLoss применит sigmoid внутри

In [138]:
# Гиперпараметры
num_labels = 14  # Число патологий в IU X-Ray (настройте по CSV)
batch_size = 16
lr = 1e-4
num_epochs = 5

In [139]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [140]:
# Инициализация модели, loss и оптимизатора
model = DenseTagger(num_labels=num_labels).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [141]:
def train_epoch(model, loader, criterion, optimizer):
    """
    Обучение модели
    """
    model.train()
    total_loss = 0.0
    for batch in loader:
        imgs = batch['img_f'].to(device)  # используем frontal view
        # Если нужны оба вида: concat или два потока
        targets = batch['labels'].to(device)  # tensor [B, num_labels]

        optimizer.zero_grad()
        logits = model(imgs)
        loss = criterion(logits, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * imgs.size(0)
    return total_loss / len(loader.dataset)

In [142]:
def validate_epoch(model, loader, criterion):
    """
    Валидация модели
    """
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for batch in loader:
            imgs = batch['img_f'].to(device)
            targets = batch['labels'].to(device)
            logits = model(imgs)
            loss = criterion(logits, targets)
            total_loss += loss.item() * imgs.size(0)
    return total_loss / len(loader.dataset)

Обучение

In [143]:
for epoch in range(1, num_epochs + 1):
    train_loss = train_epoch(model, train_dl, criterion, optimizer)
    val_loss = validate_epoch(model, test_dl, criterion)
    print(f"Epoch {epoch}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")

# Сохранение чекпойнта
torch.save(model.state_dict(), 'dense_tagger.pt')
print("Training complete. Model saved to dense_tagger.pt")

ValueError: Target size (torch.Size([16, 1635])) must be the same as input size (torch.Size([16, 14]))